# Flappy sim environment

In [ ]:
from tinysim.flappy.widget import FlappySim

env = FlappySim()
env.render()

# This is an example of a hard coded solution
actions = [a for a in [0, 0, 1, 0, 0, 1, 0, 0, 0] * 2 for _ in range(10)]

for t in range(len(actions)):
    action = actions[t]
    state = await env.step(action)
    # print(f"t={t:03d}", f"state={state}")

In [ ]:
from tinysim.flappy.tk import FlappyTkFrontend

env = FlappyTkFrontend()
env.render()

# This is an example of a hard coded solution
actions = [a for a in [0, 0, 1, 0, 0, 1, 0, 0, 0] * 2 for _ in range(10)]

for t in range(len(actions)):
    action = actions[t]
    state = await env.step(action)
    # print(f"t={t:03d}", f"state={state}")

# Now try to use reinforcement learning to solve the problem!

## Approach 1